In [1]:
# Bayesian Network example from Russell/Norvig chapter 14 (apparently taken from Judea Pearl)
# https://media.cheggcdn.com/media%2F400%2F4005f49b-e8f7-4c69-93f2-9af991a1bba8%2FphpsVaHxx.png
binvar = lambda v,p: p if v else 1-p

burglary = lambda v: binvar(v,0.001)
earthquake = lambda v: binvar(v,0.002)

def alarm(v,b,e):
    if b and e:
        p = 0.95
    elif b and not e:
        p = 0.94
    elif not b and e:
        p = 0.29
    else:
        p = 0.001
    return binvar(v,p)

def john(v,a):
    if a:
        p = 0.9
    else:
        p = 0.05
    return binvar(v,p)

def mary(v,a):
    if a:
        p = 0.7
    else:
        p = 0.01
    return binvar(v,p)

joint = lambda b,e,a,j,m: burglary(b)*earthquake(e)*alarm(a,b,e)*john(j,a)*mary(m,a)

In [2]:
import pandas as pd
import numpy as np

flag = lambda value,index: True if (value & (2**index)) > 0 else False

numberOfRows = 2**5
df = pd.DataFrame(index=np.arange(numberOfRows), columns=('burglary', 'earthquake', 'alarm', 'john', 'mary', 'probability') )
for i in np.arange(numberOfRows):
    b = flag(i,4)
    e = flag(i,3)
    a = flag(i,2)
    j = flag(i,1)
    m = flag(i,0)
    df.loc[i] = [b, e, a, j, m, joint(b,e,a,j,m)]

In [3]:
df.head()

,burglary,earthquake,alarm,john,mary,probability
0,False,False,False,False,False,0.936743
1,False,False,False,False,True,0.00946205
2,False,False,False,True,False,0.0493022
3,False,False,False,True,True,0.000498002
4,False,False,True,False,False,2.99101e-05


In [4]:
#syntax from here: https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas
a_marginal = df.loc[df['alarm'] == True, 'probability'].sum()
print('             P(Alarm) =', a_marginal)
jm_marginal = df.loc[(df['john'] == True) & (df['mary'] == True), 'probability'].sum()
print('         P(John,Mary) =', jm_marginal)
ajm_marginal = df.loc[(df['alarm'] == True) & (df['john'] == True) & (df['mary'] == True), 'probability'].sum()
print('   P(Alarm|John,Mary) =', ajm_marginal/jm_marginal)
bjm_marginal = df.loc[(df['burglary'] == True) & (df['john'] == True) & (df['mary'] == True), 'probability'].sum()
print('P(Burglary|John,Mary) =', bjm_marginal/jm_marginal)
ba_marginal = df.loc[(df['burglary'] == True) & (df['alarm'] == True), 'probability'].sum()
print('    P(Burglary|Alarm) =', ba_marginal/a_marginal)
ea_marginal = df.loc[(df['earthquake'] == True) & (df['alarm'] == True), 'probability'].sum()
print('  P(Earthquake|Alarm) =', ea_marginal/a_marginal)

             P(Alarm) = 0.002516442
         P(John,Mary) = 0.0020841002389999997
   P(Alarm|John,Mary) = 0.7606920388631079
P(Burglary|John,Mary) = 0.28417183536439294
    P(Burglary|Alarm) = 0.37355122828183607
  P(Earthquake|Alarm) = 0.23100870196889103
